In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install --upgrade torch

In [ ]:
!pip install pandas
!pip install numpy

In [ ]:
file_path = 'F:\Stratified_Data.csv'
df = pd.read_csv(file_path)

In [ ]:
df

,Unnamed: 0,app_id,app_name,review_text,review_score,review_votes,word_count,review_text_named
0,0,10,Counter-Strike,This is my very first game that I played on St...,1,0,75,Counter-Strike: This is my very first game tha...
1,1,10,Counter-Strike,About Counter-Strike Counter-Strike is a first...,1,1,201,Counter-Strike: About Counter-Strike Counter-S...
2,2,10,Counter-Strike,"It might look like a tiny Half Life mod, but l...",1,0,48,Counter-Strike: It might look like a tiny Half...
3,3,10,Counter-Strike,This game is a prime example of how a great ga...,1,0,68,Counter-Strike: This game is a prime example o...
4,4,10,Counter-Strike,it's good game when you play with you friends ...,1,0,60,Counter-Strike: it's good game when you play w...
...,...,...,...,...,...,...,...,...
504482,504482,563430,Void Pyramid,"Fantastic game, especially considering it's fr...",1,0,147,"Void Pyramid: Fantastic game, especially consi..."
504483,504483,563430,Void Pyramid,Wow. This game really takes me back to great h...,1,0,51,Void Pyramid: Wow. This game really takes me b...
504484,504484,563430,Void Pyramid,"Well balanced, several endings and manages to ...",1,0,34,"Void Pyramid: Well balanced, several endings a..."
504485,504485,563430,Void Pyramid,A Fun little dungeon RPG with some cool artwor...,1,0,71,Void Pyramid: A Fun little dungeon RPG with so...


In [ ]:
!pip install transformers

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build tokenizers


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\David\anaconda3\envs\gpu3\python.exe' 'C:\Users\David\anaconda3\envs\gpu3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\David\AppData\Local\Temp\tmpaxhn3po_'
       cwd: C:\Users\David\AppData\Local\Temp\pip-install-yannssdl\tokenizers_f69ca0e090af446ca20b056c1c0de91a
  Complete output (51 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.6
  creating build\lib.win-amd64-3.6\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-3.6\tokenizers
  creating build\lib.win-amd64-3.6\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-3.6\tokenizers\models
  creating build\lib.win-amd64-3.6\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-3.6\tokenizers\decoders
  creating build\lib.win-amd64-3.6\tokenizers\normalizers
  c

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
%%time
class ReviewsDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length=512):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = str(self.reviews.iloc[idx]['review_text_named'])
        inputs = self.tokenizer(review, return_tensors="pt", max_length=self.max_length, truncation=True)
        return inputs

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to('cuda')

# Add a new token for the start of each review
new_tokens = tokenizer.add_tokens([""])

# Resize the token embeddings in the model
model.resize_token_embeddings(len(tokenizer))

# Create a DataLoader for the dataset with a custom collate_fn
def collate_fn(batch):
    # Extract input_ids from batch and pad sequences
    inputs = pad_sequence([b["input_ids"].squeeze() for b in batch], batch_first=True)
    return {"input_ids": inputs}

dataset = ReviewsDataset(reviews=df, tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3

# Fine-tune the model
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch["input_ids"].to('cuda')

        # Model outputs are logits, no need for labels
        outputs = model(inputs)
        logits = outputs.logits

        # Assuming you have target labels in batch["labels"]
        labels = inputs.clone()

        # Compute loss using the provided labels
        loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2_model")
tokenizer.save_pretrained("fine_tuned_gpt2_model")

Epoch 1, Loss: 4.930660679747234e-07
Epoch 2, Loss: 2.93576420062891e-07
Epoch 3, Loss: 2.9552310820690764e-07
Wall time: 8h 3min 3s


('fine_tuned_gpt2_model\\tokenizer_config.json',
 'fine_tuned_gpt2_model\\special_tokens_map.json',
 'fine_tuned_gpt2_model\\vocab.json',
 'fine_tuned_gpt2_model\\merges.txt',
 'fine_tuned_gpt2_model\\added_tokens.json')

In [ ]:
model.save_pretrained(r"C:\Users\David\Downloads\ReviewGPT")
tokenizer.save_pretrained(r"C:\Users\David\Downloads\ReviewGPT_Token")

('C:\\Users\\David\\Downloads\\ReviewGPT_Token\\tokenizer_config.json',
 'C:\\Users\\David\\Downloads\\ReviewGPT_Token\\special_tokens_map.json',
 'C:\\Users\\David\\Downloads\\ReviewGPT_Token\\vocab.json',
 'C:\\Users\\David\\Downloads\\ReviewGPT_Token\\merges.txt',
 'C:\\Users\\David\\Downloads\\ReviewGPT_Token\\added_tokens.json')